In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),     # Resize images to 224x224
    transforms.ToTensor(),              # Convert images to PyTorch tensors
    transforms.Normalize(              # Normalize pixel values
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
train_dataset = torchvision.datasets.ImageFolder(root='train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='valid', transform=transform)

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=25, shuffle=False)

In [5]:
model = models.resnet18(weights=False)
num = model.fc.in_features
model.fc = nn.Linear(num, len(train_dataset.classes))

C:\Users\dared\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [7]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device outside the loop

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/10], Loss: 1.9740
Epoch [2/10], Loss: 1.4110
Epoch [3/10], Loss: 1.1637
Epoch [4/10], Loss: 1.0027
Epoch [5/10], Loss: 0.9009
Epoch [6/10], Loss: 0.8172
Epoch [7/10], Loss: 0.7559
Epoch [8/10], Loss: 0.7103
Epoch [9/10], Loss: 0.6589
Epoch [10/10], Loss: 0.6230


In [8]:
# epochs = 10
# 
# for epoch in range(epochs):
#     model.train()
#     r_loss = 0
#     for inputs, labels in train_dataset:
#         optimizer.zero_grad()
#         image = torch.unsqueeze(inputs, 0)
#         res = model(image)
#         targets = torch.tensor(labels)
#         loss = crterion(res, targets)
#         loss.backward()
#         optimizer.step()
#         r_loss += loss.item() * inputs.size(0)
#     epoch_loss = r_loss/len(train_loader.dataset)
#     print(f'Epoch [{epoch+1}/{num}], Loss: {epoch_loss:.4f}')    
torch.cuda.is_available()

True

In [9]:
model.eval()
correct = 0
total = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device outside the loop
with torch.no_grad():
    for inputs, labels in test_loader:
        
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f'Accuracy on test set: {100 * accuracy:.2f}%')

Accuracy on test set: 83.09%


NameError: name 'model' is not defined